In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score


In [7]:
#loading in the datasets
df_training = pd.read_csv('../data/archive/train.csv')
df_testing = pd.read_csv('../data/archive/test.csv')

In [8]:
#Separating X and y training and testing
x_train = df_training.iloc[:, :-2]
y_train = df_training.iloc[:,-1]

x_test = df_testing.iloc[:, :-2]
y_test = df_testing.iloc[:,-1]

In [9]:
#getting all of our randomized samples from the training
all_samples = []

for sample in range(0,5):
    all_samples.append(df_training.sample(n=5000, replace= True))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
#initializing our lists so we can later store the best params that achieve best scores across metrics
accuracy_best_params = []
f1_best_params = []
precision_best_params = []



for sample in all_samples:
    
    xgb = XGBClassifier()
    
    #Making a grid of values we want our grid search to test to find the best parameters
    grid_values =[{'n_estimators': [10, 100, 1000], 'learning_rate': [0.15, 0.2, 0.1, 0.01], 'max_depth':list(range(1,20,4))}]


    xgb_classifier = RandomizedSearchCV(estimator = xgb, param_distributions
                                        = grid_values, cv = 5,
                      scoring = ['accuracy', 'f1_micro', 'precision_micro'], refit = False, verbose = 0)
    
    x_train = sample.iloc[:, :-2]
    y_train = sample.iloc[:,-1]
    
    xgb_model = xgb_classifier.fit(x_train, y_train)

    accuracy_best_params.append(xgb_model.cv_results_['params'][ np.argmin(xgb_model.cv_results_['rank_test_accuracy'])])
    f1_best_params.append(xgb_model.cv_results_['params'][ np.argmin(xgb_model.cv_results_['rank_test_f1_micro'])])
    precision_best_params.append(xgb_model.cv_results_['params'][ np.argmin(xgb_model.cv_results_['rank_test_precision_micro'])])

In [13]:
accuracy_best_params

[{'n_estimators': 1000, 'max_depth': 5, 'learning_rate': 0.1}]

In [11]:
accuracy_test_score = []

i = 0
for param in accuracy_best_params:

    xgb_clf = XGBClassifier(n_estimators = accuracy_best_params[i].get('n_estimators'),
                            learning_rate = accuracy_best_params[i].get('learning_rate'),
                           max_depth = accuracy_best_params[i].get('max_depth'))
    model = xgb_clf.fit(x_train, y_train)

    y_pred = model.predict(x_test)

    accuracy_test_score.append(accuracy_score(y_test, y_pred))
    
    i += 1

In [12]:
accuracy_test_score

[0.9419748897183576]

In [ ]:
f1_test_score = []

i = 0
for param in f1_best_params:

    xgb_clf = XGBClassifier(n_estimators = f1_best_params[i].get('n_estimators'),
                            learning_rate = f1_best_params[i].get('learning_rate'),
                           max_depth = f1_best_params[i].get('max_depth'))
    model = xgb_clf.fit(x_train, y_train)

    y_pred = model.predict(x_test)

    f1_test_score.append(f1_score(y_test, y_pred), average = 'micro')
    
    i += 1

In [ ]:
f1_test_score

In [ ]:
precision_test_score = []

i = 0
for param in precision_best_params:

    xgb_clf = XGBClassifier(n_estimators = precision_best_params[i].get('n_estimators'),
                            learning_rate = precision_best_params[i].get('learning_rate'),
                           max_depth = precision_best_params[i].get('max_depth'))
    model = xgb_clf.fit(x_train, y_train)

    y_pred = model.predict(x_test)

    precision_test_score.append(precision_score(y_test, y_pred), average = 'micro')
    
    i += 1

In [ ]:
precision_test_score